In [101]:
# !pip install implicit

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [103]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [104]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [105]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [106]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights.item_id, size=n, replace=False, p=items_weights.weight)
    
    return recs.tolist()

In [107]:
%%time

# your_code
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value': 'weight'})
items_weights.loc[items_weights['weight'] < 1, 'weight'] = 1
items_weights['weight'] = np.log(items_weights['weight'])
items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()

Wall time: 124 ms


In [108]:
items_weights.weight.sum()

0.9999999999999999

In [109]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12352291, 1053464, 1066796, 1066869, 1131526]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1004743, 953712, 914777, 408450, 1008787]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5, Precision@3, Recall@5 для каждого алгоритма с помощью функций из вебинара 1. Какой алгоритм показывает лучшее качество?

In [110]:
result = pd.read_pickle('predictions_basic.pkl')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6602611, 836580, 879008, 9682126, 1859610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[1081177, 995785, 1004906, 1082185, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[848356, 12131837, 6513993, 1383142, 9194302]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[1081177, 995785, 1004906, 1082185, 1029743]"


In [111]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [112]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

#### Precision@5

In [113]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k = 5), axis=1).mean(),4)}:{name_col}")

0.0009:random_recommendation
0.1552:popular_recommendation
0.1457:itemitem
0.1355:cosine
0.1355:tfidf
0.1623:own_purchases


#### Precision@3

In [114]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k = 3), axis=1).mean(),4)}:{name_col}")

0.001:random_recommendation
0.1378:popular_recommendation
0.1197:itemitem
0.0589:cosine
0.0589:tfidf
0.0875:own_purchases


#### Recall@5

In [115]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: recall_at_k(row[name_col], row['actual'], k = 5), axis=1).mean(),4)}:{name_col}")

0.0001:random_recommendation
0.025:popular_recommendation
0.0162:itemitem
0.014:cosine
0.014:tfidf
0.0183:own_purchases


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 (или другое количество) товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [31]:
# your_code